## Генерация выборок


In [203]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

def gen_gaussian_lin_mono(mu, sigma, n):
    x = np.random.normal(mu, sigma, n)
    z = np.random.normal(mu, sigma, n)
    y = 0.6 * x + 0.4 * z
    return x, y

def gen_notgaussian_notlin_mono(mu, sigma, n):
    x = np.random.normal(mu, sigma, n)
    z = np.random.normal(mu*2, sigma*10, n)
    y = x**3 + z
    return x, y

def gen_notgaussian_notlin_notmono(mu, sigma, n):
    x = np.random.normal(mu, sigma, n)
    z = np.random.normal(mu*2, sigma*2, n)
    y = x ** 2 + z**3*z ** 2 + x**4 + np.sin(z)
    return x, y

In [162]:
def make_test(test, count_view= 100, n =100):
    freq_error_gaussian = 0
    freq_error_notgaussian_liner = 0
    freq_error_notgaussian_notliner = 0
    
    for _ in range(n):
        x_gaus, y_gaus = gen_gaussian_lin_mono(2, 10, count_view)
        x_notgaus_liner, y_notgaus_liner = gen_notgaussian_notlin_mono(2, 10, count_view)
        x_notgaus_notliner, y_notgaus_notliner = gen_notgaussian_notlin_notmono(2, 10, count_view)
        freq_error_gaussian = freq_error_gaussian + test(x_gaus,y_gaus)[2]
        freq_error_notgaussian_liner =freq_error_notgaussian_liner + test(x_notgaus_liner,y_notgaus_liner)[2]
        freq_error_notgaussian_notliner =freq_error_notgaussian_notliner + test(x_notgaus_notliner,y_notgaus_notliner)[2]
    
    return freq_error_gaussian/n, freq_error_notgaussian_liner/n, freq_error_notgaussian_notliner/n
        
        
    

## Выборочный коэффициент корреляции

Проверка зависимости двух нормально распределённых признаков

In [120]:
def corr_test(x, y, alpha = 0.05):
    # флаг, который указывает, принимаем H0 или отвергаем : 1-отвергаем, 0 - принимаем
    complete = 0
    # кол-во элементов в выборке
    n = len(x)
    # вычисялем выборочное среднее выборок
    x_mean = x.mean()
    y_mean = y.mean()
    # вычисляем выборочный коэф корреляции
    r = (1 / n * np.sum((x - x_mean) * (y - y_mean))) / \
    (1 / n * np.sum((x - x_mean) ** 2) * 1 / n * np.sum((y - y_mean) ** 2)) ** 0.5
    t_value = n ** 0.5 * r
    p_value = np.min(np.array([2 * stats.norm.cdf(t_value), 2 - 2 * stats.norm.cdf(t_value)]), axis=0)
    if (p_value > alpha / 2) and (p_value < (1 - alpha / 2)):
        complete = 0
    else:
        complete = 1
    return r, t_value, p_value, complete

In [121]:
f_gaus, f_ngaus, g_ngaus_nl = make_test(corr_test)
print(f"""
    Частота ошибок критерия корреляции
    Гауссова {f_gaus}
    Не гауссова линейная {f_ngaus}
    Не гаусова не линейная {g_ngaus_nl}
""")


    Частота ошибок критерия корреляции
    Гауссова 6.008527009271347e-15
    Не гауссова линейная 0.45488745295778094
    Не гаусова не линейная 0.4830027025300453



/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in square
  # This is added back by InteractiveShellApp.init_path()


## Критерий Спирмана

Признаки ненормальны, подозревается монотонная зависимость

In [87]:
def ranging(x):
    x_sorted = np.sort(x)
    R = np.array([])
    elems = np.array([])
    x_rangs =  np.array([])
    i = 1
    while i <= len(x):
        x_count = (x_sorted == x_sorted[i-1]).sum()
        if x_count == 1:
            R = np.append(R, i)
            elems = np.append(elems, x_sorted[x_sorted == x_sorted[i-1]])
        else:
            for j in range(x_count):
                R = np.append(R, i + (c - 1) / 2)
                elems = np.append(elems, x_sorted[x_sorted == x_sorted[i-1]][0])
        i += x_count
    for i in range(len(x)):
        idx = np.where(elems == x[i])
        x_rangs = np.append(x_rangs, R[idx[0]])
    return x_rangs

def spearman(x, y, alpha = 0.05):
    complete = 0
    n = len(x)
    x_rangs = ranging(x)
    y_rangs = ranging(y)
    r_spearman = 1 - 6 / (n ** 3 - n) * np.sum((x_rangs - y_rangs) ** 2)
    t_value = (n-1) ** 0.5 * r_spearman
    p_value = np.min(np.array([2 * stats.norm.cdf(t_value), 2 - 2 * stats.norm.cdf(t_value)]), axis=0)
    if (p_value > alpha / 2) and (p_value < (1 - alpha / 2)):
        complete = 0
    else:
        complete = 1
    return r_spearman, t_value, p_value, complete

In [88]:
from scipy import stats

In [89]:
f_gaus, f_ngaus, g_ngaus_nl = make_test(spearman)
print(f"""
    Частота ошибок критерия Спирмена
    Гауссова {f_gaus}
    Не гауссова линейная {f_ngaus}
    Не гаусова не линейная {g_ngaus_nl}
""")


    Частота ошибок критерия Спирмена
    Гауссова 6.632472349110685e-15
    Не гауссова линейная 0.0501434547555514
    Не гаусова не линейная 0.28512957310432746



## Критерий Хи-квадрат Пирсона

In [167]:
def to_nominal(x, k):
    x_sort = np.sort(x)
    x_in_scale = np.zeros(k)
    slices = np.linspace(x_sort[0], x_sort[-1], k+1)
    for i in range(k):
        x_part = x[x >= slices[i]]
        x_count = (x_part <= slices[i+1]).sum()
        x_in_scale[i] = x_count
    return x_in_scale

def pirson(x, y, alpha = 0.05):
    # флаг, который указывает, принимаем H0 или отвергаем : 1-отвергаем, 0 - принимаем
    complete = 0
    # кол-во элементов в выборке
    n = len(x)
    # переводим в номинальную шкалу
    x_nom = to_nominal(x, 5)
    y_nom = to_nominal(y, 5)
    matrix = np.concatenate(([x_nom], [y_nom])) 
#     print('Contingency matrix:\n')
#     print(matrix,'\n')
    chi2_value = 0
    for i in range(2):
        for j in range(len(x_nom)):
            chi2_value += (matrix[i][j] - (np.sum(matrix[i]) * np.sum(matrix[:,j]) / np.sum(matrix))) ** 2 \
            / (np.sum(matrix[i]) * np.sum(matrix[:,j]) / np.sum(matrix))
            
    # вычисляем p-value
    pirson_coeff = (chi2_value / (np.sum(matrix) + chi2_value)) ** 0.5
    kramer_coeff = (chi2_value / (np.sum(matrix) * np.min([(len(x_nom) - 1), 1]))) ** 0.5
    chuprov_coeff =  (chi2_value / (np.sum(matrix) * ((len(x_nom) - 1)) ** 0.5)) ** 0.5
#     print(f'Pirson coeff = {pirson_coeff}\n')
#     print(f'Kramer coeff = {kramer_coeff}\n')
#     print(f'Chuprov coeff = {chuprov_coeff}\n')
    p_value = 1 - stats.chi2.cdf(chi2_value, (len(x_nom) - 1))
    # принимаем H0 или отвергаем
    if p_value > alpha:
        complete = 0
    else:
        complete = 1
    return chi2_value, p_value, complete

In [172]:
def make_test(test, count_view= 100, n =100):
    freq_error_gaussian = 0
    freq_error_notgaussian_liner = 0
    freq_error_notgaussian_notliner = 0
    
    for _ in range(n):
        x_gaus, y_gaus = gen_gaussian_lin_mono(100, 2, count_view)
        x_notgaus_liner, y_notgaus_liner = gen_notgaussian_notlin_mono(100, 2, count_view)
        x_notgaus_notliner, y_notgaus_notliner = gen_notgaussian_notlin_notmono(100, 2, count_view)
        freq_error_gaussian = freq_error_gaussian + test([x_gaus,y_gaus])[1] <= 0.05
        freq_error_notgaussian_liner =freq_error_notgaussian_liner + test([x_notgaus_liner,y_notgaus_liner])[1] <=0.0005
        freq_error_notgaussian_notliner =freq_error_notgaussian_notliner + test([x_notgaus_notliner,y_notgaus_notliner])[1]<=0.0005
    
    return freq_error_gaussian/n, freq_error_notgaussian_liner/n, freq_error_notgaussian_notliner/n
        
        
    

In [200]:
def count2(ax,bx,ay,by,x,y):
    count = 0
    for i in range(0,len(x)):
        if(x[i] >= ax)and(x[i] <= bx):
            if(y[i] >= ay)and(y[i] <= by):
                count += 1
    return count
def count(a,b,x):
    count = 0
    for i in range(len(x)):
        if(x[i] >= a)and(x[i] <= b):
            count += 1
    return count

def hi2(x,y):
    n = len(x)
    ax = x.min()
    bx = x.max()
    ay = y.min()
    by = y.max()
    k = 5
    xk = np.zeros((k+1))
    xk[0] = ax
    yk = np.zeros((k+1))
    yk[0] = ay
    for i in range(1,k+1):
        xk[i] = xk[i-1] + (bx-ax)/k
        yk[i] = yk[i-1] + (by-ay)/k
    ni = np.zeros((k))
    nj = np.zeros((k))
    for i in range(0,k):
        ni[i] = count(xk[i],xk[i+1],x)#кол-во измерений в интервале xk[i], xk[i+1]
        nj[i] = count(yk[i],yk[i+1],y)#кол-во измерений в интервале yk[i], yk[i+1]
    nij = np.zeros((k,k))
    for i in range(0,k):
        for j in range(0,k):#кол-во объектов из интервалов xk[i],xk[i+1] и yk[j],yk[j+1]
            nij[i][j] = count2(xk[i],xk[i+1],yk[j],yk[j+1],x,y)
    ni_ = np.zeros((k))
    nj_ = np.zeros((k))
    for i in range(0,k):
        for j in range(0,k):
            ni_[i] += nij[i][j]
    for i in range(0,k):
        for j in range(0,k):
            nj_[i] += nij[j][i]
    krit1 = 0
    #print(nij)
    if(0 in ni_):
        return -1
    if(0 in nj_):
        return -1
    for i in range(0,k):
        for j in range(0,k):
            krit1 += ((nij[i][j] - (ni_[i]*nj_[j]/n))**2)/(ni_[i]*nj_[j]/n)
    #print(krit1)
    # print("||||||||||||||||||||||||||||||")
    # print("Коэфф. С.К. Напр.",np.sqrt(krit1/n))
    # print("Коэфф. Пирсона",np.sqrt(krit1/(n+krit1)))
    # print("Коэфф. Чупрова",np.sqrt(krit1/(n*np.sqrt((k-1)*(k-1)))))
    # print("Коэфф. Крамера",np.sqrt(krit1/(n*(k-1))))
    if(krit1 < 10.522):#101.89 124.34
        return 1#если независимы
    else:
        return 0#если зависимы


In [204]:
def make_test(test, count_view= 100, n =100):
    freq_error_gaussian = 0
    freq_error_notgaussian_liner = 0
    freq_error_notgaussian_notliner = 0
    
    for _ in range(n):
        x_gaus, y_gaus = gen_gaussian_lin_mono(100, 2, count_view)
        x_notgaus_liner, y_notgaus_liner = gen_notgaussian_notlin_mono(100, 2, count_view)
        x_notgaus_notliner, y_notgaus_notliner = gen_notgaussian_notlin_notmono(100, 2, count_view)
        freq_error_gaussian = freq_error_gaussian + test(x_gaus,y_gaus)
        freq_error_notgaussian_liner =freq_error_notgaussian_liner + test(x_notgaus_liner,y_notgaus_liner)
        freq_error_notgaussian_notliner =freq_error_notgaussian_notliner + test(x_notgaus_notliner,y_notgaus_notliner)
    
    return freq_error_gaussian/n, freq_error_notgaussian_liner/n, freq_error_notgaussian_notliner/n
        
        
    

In [206]:
f_gaus, f_ngaus, g_ngaus_nl = make_test(hi2)
print(f"""
    Частота ошибок критерия Хи-2
    Гауссова {abs(f_gaus)}
    Не гауссова линейная {abs(f_ngaus)}
    Не гаусова не линейная {abs(g_ngaus_nl)}
""")


    Частота ошибок критерия Хи-2
    Гауссова 0.02
    Не гауссова линейная 0.02
    Не гаусова не линейная 0.01

